# Overfitting Experimentation

In model_selection.ipynb we selected convolutional neural networks as our model class and identified overfitting as a major issue with this dataset. 
In this notebook we experiment with different ways to configure our model for reducing overfitting and with including more variety into our data by using data generators.

We do this in two independent experiments. 




. These are:
- Dropout Layers
- Batch Normalization
- Regularization
- weight constraints

In addition, we experiment with data generators to include more variety in our dataset.




In this notebook we carry out experiments to select which machine learning model 

- feature engineering
- proportional train test split
- data Generator
- dropout
- batch normalization
- statistical tests

In [1]:
# correct working directory. But only once. 
if not "working_directory_corrected" in vars():
    %cd ..
    working_directory_corrected = True

import numpy as np
import matplotlib.pyplot as plt 
import tensorflow as tf

from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential  # Model type to be used
from keras.layers import (
    Activation, # Types of layers to be used in our model
    Conv2D,
    Conv3D,
    Dense,  # Types of layers to be used in our model
    Flatten,
    GlobalAveragePooling2D,
    MaxPooling2D,
    MaxPooling3D,
    ZeroPadding2D,
)

from models.CNN_Builder import CNNBuilder
from evaluation.multi_run_evaluation import MultiRunEvaluation
from evaluation.evaluation_metrics import EvaluationMetrics
from data.dataset import Dataset

dataset = Dataset()

/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project


2023-01-03 15:02:33.056018: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Data Loading

Before carrying out the experiments, let's load our dataset.

In addition to loading our dataset we will use image generators for the remainder of the experiment. For now, we will generate dummy image generators that don't include any changes.

In [2]:
X, y = dataset.get_prepared_data()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
y_train_prob = np.array([y_train[i] for i in range(len(y_train))])
train_datagen = ImageDataGenerator(
    horizontal_flip=False,
    vertical_flip=False,
    fill_mode="nearest"
)
train_generator = train_datagen.flow(x=X_train, y=y_train_prob)

## Experiment 1: Network Configuration

In this experiment we measure the influence of network configurations on generalization. For this purpose we test the following configurations:
- Dropout Layers
- Batch Normalization
- Regularization
- weight constraints

### Experiment Setup:
For our experiment we learn the network identified in previous experiments once with each of the configuration parameters and without and compare the hamming score.

As we have noted in past experiments, the variance of different weight initializations is a concern. For this reason, we implemented the class *Multi_Run_Evaluaton* which can run an experiment multiple times, recording the evaluation metrics and calculating minimum, maximum, mean and standard deviation. 

To make the configuration code less verbous and less error prone we also impemented a builder lass that assembles the CNN. 

During our experiment we ran the cell below with different values for *apply_regularization*, *apply_dropout*, *apply_batch_normalization* and *weight_constraints" and recorded results in the cell below. 

In [3]:
cnn_builder = CNNBuilder(
    convolutional_layers=[16],
    fully_connected_layers=[100],
    in_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]),    # type: ignore
    out_shape=y_train.shape[1]  # type: ignore
)
cnn_builder.apply_regularization = True
cnn_builder.apply_dropout = True
cnn_builder.apply_batch_normalization = True
cnn_builder.weight_constraints = True

evaluator = MultiRunEvaluation(model_creation=cnn_builder.build_model)
evaluator.evaluate(
    nr_runs=10,
    epochs=100,
    early_stopping_patience=5,
    train_generator= train_generator,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test
)
evaluator.print_metrics()


Running experiment 1 of, 10


2023-01-03 15:02:39.518155: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/100


/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 6s - loss: 2.1905 - val_loss: 2.3641 - 6s/epoch - 263ms/step
Epoch 2/100
21/21 - 5s - loss: 2.1849 - val_loss: 2.3904 - 5s/epoch - 251ms/step
Epoch 3/100
21/21 - 5s - loss: 2.1300 - val_loss: 2.2889 - 5s/epoch - 230ms/step
Epoch 4/100
21/21 - 5s - loss: 2.0662 - val_loss: 2.2136 - 5s/epoch - 250ms/step
Epoch 5/100
21/21 - 5s - loss: 1.9981 - val_loss: 2.1762 - 5s/epoch - 247ms/step
Epoch 6/100
21/21 - 5s - loss: 1.9165 - val_loss: 2.0583 - 5s/epoch - 240ms/step
Epoch 7/100
21/21 - 5s - loss: 1.8500 - val_loss: 1.9977 - 5s/epoch - 233ms/step
Epoch 8/100
21/21 - 5s - loss: 1.8002 - val_loss: 1.9161 - 5s/epoch - 235ms/step
Epoch 9/100
21/21 - 5s - loss: 1.7687 - val_loss: 1.8344 - 5s/epoch - 239ms/step
Epoch 10/100
21/21 - 5s - loss: 1.7476 - val_loss: 1.7632 - 5s/epoch - 240ms/step
Epoch 11/100
21/21 - 5s - loss: 1.7404 - val_loss: 1.7525 - 5s/epoch - 245ms/step
Epoch 12/100
21/21 - 5s - loss: 1.7328 - val_loss: 1.7423 - 5s/epoch - 248ms/step
Epoch 13/100
21/21 - 5s - loss: 1.728

...

21/21 - 7s - loss: 2.1461 - val_loss: 2.2141 - 7s/epoch - 332ms/step
Epoch 2/100
21/21 - 7s - loss: 2.1515 - val_loss: 2.1527 - 7s/epoch - 324ms/step
Epoch 3/100
21/21 - 8s - loss: 2.0926 - val_loss: 2.1155 - 8s/epoch - 374ms/step
Epoch 4/100
21/21 - 6s - loss: 2.0337 - val_loss: 2.0749 - 6s/epoch - 289ms/step
Epoch 5/100
21/21 - 5s - loss: 1.9710 - val_loss: 2.0697 - 5s/epoch - 249ms/step
Epoch 6/100
21/21 - 5s - loss: 1.8948 - val_loss: 1.9918 - 5s/epoch - 239ms/step
Epoch 7/100
21/21 - 5s - loss: 1.8311 - val_loss: 1.9141 - 5s/epoch - 245ms/step
Epoch 8/100
21/21 - 5s - loss: 1.7871 - val_loss: 1.8495 - 5s/epoch - 254ms/step
Epoch 9/100
21/21 - 6s - loss: 1.7587 - val_loss: 1.8034 - 6s/epoch - 278ms/step
Epoch 10/100
21/21 - 7s - loss: 1.7444 - val_loss: 1.8246 - 7s/epoch - 335ms/step
Epoch 11/100
21/21 - 7s - loss: 1.7347 - val_loss: 1.7964 - 7s/epoch - 323ms/step
Epoch 12/100
21/21 - 5s - loss: 1.7297 - val_loss: 1.7752 - 5s/epoch - 241ms/step
Epoch 13/100
21/21 - 5s - loss: 1.730

/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 5s - loss: 2.1918 - val_loss: 2.2254 - 5s/epoch - 253ms/step
Epoch 2/100
21/21 - 5s - loss: 2.1660 - val_loss: 2.2279 - 5s/epoch - 238ms/step
Epoch 3/100
21/21 - 5s - loss: 2.1187 - val_loss: 2.2260 - 5s/epoch - 250ms/step
Epoch 4/100
21/21 - 5s - loss: 2.0615 - val_loss: 2.1850 - 5s/epoch - 249ms/step
Epoch 5/100
21/21 - 5s - loss: 2.0042 - val_loss: 2.1297 - 5s/epoch - 251ms/step
Epoch 6/100
21/21 - 5s - loss: 1.9301 - val_loss: 2.0910 - 5s/epoch - 246ms/step
Epoch 7/100
21/21 - 5s - loss: 1.8534 - val_loss: 2.0075 - 5s/epoch - 256ms/step
Epoch 8/100
21/21 - 6s - loss: 1.8007 - val_loss: 1.9246 - 6s/epoch - 264ms/step
Epoch 9/100
21/21 - 5s - loss: 1.7700 - val_loss: 1.8682 - 5s/epoch - 243ms/step
Epoch 10/100
21/21 - 5s - loss: 1.7536 - val_loss: 1.8290 - 5s/epoch - 245ms/step
Epoch 11/100
21/21 - 5s - loss: 1.7421 - val_loss: 1.9013 - 5s/epoch - 244ms/step
Epoch 12/100
21/21 - 5s - loss: 1.7339 - val_loss: 1.8543 - 5s/epoch - 260ms/step
Epoch 13/100
21/21 - 5s - loss: 1.729

/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 5s - loss: 2.1501 - val_loss: 2.5161 - 5s/epoch - 237ms/step
Epoch 2/100
21/21 - 5s - loss: 2.1422 - val_loss: 2.4427 - 5s/epoch - 233ms/step
Epoch 3/100
21/21 - 5s - loss: 2.0801 - val_loss: 2.3630 - 5s/epoch - 253ms/step
Epoch 4/100
21/21 - 6s - loss: 2.0200 - val_loss: 2.2726 - 6s/epoch - 286ms/step
Epoch 5/100
21/21 - 5s - loss: 1.9488 - val_loss: 2.1499 - 5s/epoch - 255ms/step
Epoch 6/100
21/21 - 6s - loss: 1.8706 - val_loss: 2.0375 - 6s/epoch - 279ms/step
Epoch 7/100
21/21 - 7s - loss: 1.8085 - val_loss: 1.9166 - 7s/epoch - 332ms/step
Epoch 8/100
21/21 - 6s - loss: 1.7714 - val_loss: 1.8427 - 6s/epoch - 277ms/step
Epoch 9/100
21/21 - 7s - loss: 1.7482 - val_loss: 1.8127 - 7s/epoch - 318ms/step
Epoch 10/100
21/21 - 7s - loss: 1.7377 - val_loss: 1.7817 - 7s/epoch - 320ms/step
Epoch 11/100
21/21 - 7s - loss: 1.7326 - val_loss: 1.7649 - 7s/epoch - 322ms/step
Epoch 12/100
21/21 - 6s - loss: 1.7274 - val_loss: 1.7621 - 6s/epoch - 280ms/step
Epoch 13/100
21/21 - 6s - loss: 1.727

### Results

Running our Experiment yielded the following results:

| Configuration          | Min  | Max  | Mean  | St. Dev |
| ---------------------- | ---- | ---- | ----- | ------- |
| No Configuration       | 0.17 | 0.21 | 0.195 | 0.012   |
| Regularization         | 0.16 | 0.21 | 0.184 | 0.017   |
| Dropout                | 0.17 | 0.21 | 0.19  | 0.011   |
| Batch Normalization    | 0.02 | 0.16 | 0.109 | 0.040   |
| Weight Constraints     | 0.11 | 0.20 | 0.163 | 0.029   |

### Interpretation
None of the configurations have improved the Hamming score of the test set. While regularization and Dropout are close - and thus could be a result of random variation, batch normalization led to a significantly worse performance. 

This does not necessarily mean, that these configurations are not useful. It may mean that their effect is overshadowed by the general issue of having a very small data set. In previous experiments we have seen dropout increase the hammilton score slightly, so there seems to be some promise. 

For now, this experiment is inconclusive. we will revisit this experiment after increasing the data set complexity with data generators, hoping that this will increase the complexity enough for these regularizations to make a consistent difference. 



## Experiment 2: Image Generators

In this experiment we introduce variabilty into the training set by using image generators. We hope that this increases the variability of input seen by the model and helps it generalize. 

Specifically, we try to:
- Mirror images horizonzally
- Rotate the image up to 30 degrees in either direction
- Shift the image by 10% of the image size in horizontal or vertical direction
- change brightness by 20%
- zooming in or out by 20%

### Experiment Setup:
For our experiment we learn the network identified in previous experiments once with each of the configuration parameters and without and compare the hamming score.

As in the previoius experiment, we use the CNN Factory and Multi Run Evaluation.
During our experiment we ran the cell below with different convigurations of the train data generator and recorded results in the cell below. 

In [4]:
train_datagen = ImageDataGenerator(
    #rotation_range=30,
    #zoom_range = 0.20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    # brightness_range= [0.8, 1.2],
    #horizontal_flip=True,
    fill_mode="nearest"
)
train_generator = train_datagen.flow(x=X_train, y=y_train_prob)

cnn_builder = CNNBuilder(
    convolutional_layers=[16],
    fully_connected_layers=[100],
    in_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]),  # type: ignore
    out_shape=y_train.shape[1]  # type: ignore
)

evaluator = MultiRunEvaluation(model_creation=cnn_builder.build_model)
evaluator.evaluate(
    nr_runs=10,
    epochs=100,
    early_stopping_patience=5,
    train_generator= train_generator,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test
)
evaluator.print_metrics()


Running experiment 1 of, 10
Epoch 1/100


/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 5s - loss: 0.3134 - val_loss: 0.2849 - 5s/epoch - 231ms/step
Epoch 2/100
21/21 - 5s - loss: 0.2810 - val_loss: 0.2813 - 5s/epoch - 246ms/step
Epoch 3/100
21/21 - 5s - loss: 0.2790 - val_loss: 0.2813 - 5s/epoch - 236ms/step
Epoch 4/100
21/21 - 4s - loss: 0.2767 - val_loss: 0.2793 - 4s/epoch - 209ms/step
Epoch 5/100
21/21 - 4s - loss: 0.2750 - val_loss: 0.2754 - 4s/epoch - 210ms/step
Epoch 6/100
21/21 - 4s - loss: 0.2725 - val_loss: 0.2743 - 4s/epoch - 207ms/step
Epoch 7/100
21/21 - 5s - loss: 0.2672 - val_loss: 0.2736 - 5s/epoch - 215ms/step
Epoch 8/100
21/21 - 5s - loss: 0.2656 - val_loss: 0.2723 - 5s/epoch - 258ms/step
Epoch 9/100
21/21 - 5s - loss: 0.2626 - val_loss: 0.2678 - 5s/epoch - 220ms/step
Epoch 10/100
21/21 - 5s - loss: 0.2599 - val_loss: 0.2692 - 5s/epoch - 228ms/step
Epoch 11/100
21/21 - 5s - loss: 0.2580 - val_loss: 0.2669 - 5s/epoch - 240ms/step
Epoch 12/100
21/21 - 5s - loss: 0.2536 - val_loss: 0.2691 - 5s/epoch - 226ms/step
Epoch 13/100
21/21 - 5s - loss: 0.250

/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 5s - loss: 0.3207 - val_loss: 0.2844 - 5s/epoch - 258ms/step
Epoch 2/100
21/21 - 4s - loss: 0.2803 - val_loss: 0.2822 - 4s/epoch - 209ms/step
Epoch 3/100
21/21 - 4s - loss: 0.2775 - val_loss: 0.2791 - 4s/epoch - 208ms/step
Epoch 4/100
21/21 - 5s - loss: 0.2746 - val_loss: 0.2783 - 5s/epoch - 241ms/step
Epoch 5/100
21/21 - 4s - loss: 0.2713 - val_loss: 0.2742 - 4s/epoch - 213ms/step
Epoch 6/100
21/21 - 5s - loss: 0.2695 - val_loss: 0.2733 - 5s/epoch - 223ms/step
Epoch 7/100
21/21 - 5s - loss: 0.2665 - val_loss: 0.2714 - 5s/epoch - 228ms/step
Epoch 8/100
21/21 - 5s - loss: 0.2634 - val_loss: 0.2711 - 5s/epoch - 222ms/step
Epoch 9/100
21/21 - 4s - loss: 0.2611 - val_loss: 0.2668 - 4s/epoch - 209ms/step
Epoch 10/100
21/21 - 4s - loss: 0.2587 - val_loss: 0.2649 - 4s/epoch - 210ms/step
Epoch 11/100
21/21 - 4s - loss: 0.2566 - val_loss: 0.2658 - 4s/epoch - 209ms/step
Epoch 12/100
21/21 - 4s - loss: 0.2546 - val_loss: 0.2653 - 4s/epoch - 210ms/step
Epoch 13/100
21/21 - 5s - loss: 0.250

...

21/21 - 4s - loss: 0.3359 - val_loss: 0.2892 - 4s/epoch - 208ms/step
Epoch 2/100
21/21 - 4s - loss: 0.2820 - val_loss: 0.2851 - 4s/epoch - 208ms/step
Epoch 3/100
21/21 - 4s - loss: 0.2767 - val_loss: 0.2813 - 4s/epoch - 206ms/step
Epoch 4/100
21/21 - 4s - loss: 0.2738 - val_loss: 0.2758 - 4s/epoch - 203ms/step
Epoch 5/100
21/21 - 4s - loss: 0.2701 - val_loss: 0.2737 - 4s/epoch - 206ms/step
Epoch 6/100
21/21 - 4s - loss: 0.2661 - val_loss: 0.2702 - 4s/epoch - 204ms/step
Epoch 7/100
21/21 - 4s - loss: 0.2627 - val_loss: 0.2665 - 4s/epoch - 203ms/step
Epoch 8/100
21/21 - 4s - loss: 0.2582 - val_loss: 0.2696 - 4s/epoch - 211ms/step
Epoch 9/100
21/21 - 4s - loss: 0.2563 - val_loss: 0.2635 - 4s/epoch - 206ms/step
Epoch 10/100
21/21 - 4s - loss: 0.2528 - val_loss: 0.2613 - 4s/epoch - 205ms/step
Epoch 11/100
21/21 - 4s - loss: 0.2489 - val_loss: 0.2625 - 4s/epoch - 203ms/step
Epoch 12/100
21/21 - 4s - loss: 0.2467 - val_loss: 0.2637 - 4s/epoch - 203ms/step
Epoch 13/100
21/21 - 4s - loss: 0.245

/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 4s - loss: 0.3228 - val_loss: 0.2892 - 4s/epoch - 205ms/step
Epoch 2/100
21/21 - 4s - loss: 0.2767 - val_loss: 0.2804 - 4s/epoch - 205ms/step
Epoch 3/100
21/21 - 4s - loss: 0.2730 - val_loss: 0.2785 - 4s/epoch - 210ms/step
Epoch 4/100
21/21 - 4s - loss: 0.2687 - val_loss: 0.2739 - 4s/epoch - 204ms/step
Epoch 5/100
21/21 - 4s - loss: 0.2657 - val_loss: 0.2751 - 4s/epoch - 202ms/step
Epoch 6/100
21/21 - 4s - loss: 0.2630 - val_loss: 0.2727 - 4s/epoch - 203ms/step
Epoch 7/100
21/21 - 4s - loss: 0.2588 - val_loss: 0.2734 - 4s/epoch - 204ms/step
Epoch 8/100
21/21 - 4s - loss: 0.2573 - val_loss: 0.2731 - 4s/epoch - 203ms/step
Epoch 9/100
21/21 - 4s - loss: 0.2536 - val_loss: 0.2720 - 4s/epoch - 202ms/step
Epoch 10/100
21/21 - 4s - loss: 0.2524 - val_loss: 0.2684 - 4s/epoch - 202ms/step
Epoch 11/100
21/21 - 4s - loss: 0.2500 - val_loss: 0.2667 - 4s/epoch - 205ms/step
Epoch 12/100
21/21 - 4s - loss: 0.2483 - val_loss: 0.2669 - 4s/epoch - 204ms/step
Epoch 13/100
21/21 - 4s - loss: 0.246

/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 5s - loss: 0.3137 - val_loss: 0.2854 - 5s/epoch - 216ms/step
Epoch 2/100
21/21 - 4s - loss: 0.2786 - val_loss: 0.2814 - 4s/epoch - 210ms/step
Epoch 3/100
21/21 - 4s - loss: 0.2778 - val_loss: 0.2793 - 4s/epoch - 206ms/step
Epoch 4/100
21/21 - 4s - loss: 0.2737 - val_loss: 0.2761 - 4s/epoch - 205ms/step
Epoch 5/100
21/21 - 4s - loss: 0.2693 - val_loss: 0.2725 - 4s/epoch - 207ms/step
Epoch 6/100
21/21 - 4s - loss: 0.2662 - val_loss: 0.2714 - 4s/epoch - 208ms/step
Epoch 7/100
21/21 - 5s - loss: 0.2646 - val_loss: 0.2703 - 5s/epoch - 241ms/step
Epoch 8/100
21/21 - 5s - loss: 0.2594 - val_loss: 0.2663 - 5s/epoch - 225ms/step
Epoch 9/100
21/21 - 5s - loss: 0.2565 - val_loss: 0.2658 - 5s/epoch - 225ms/step
Epoch 10/100
21/21 - 5s - loss: 0.2550 - val_loss: 0.2665 - 5s/epoch - 229ms/step
Epoch 11/100
21/21 - 5s - loss: 0.2507 - val_loss: 0.2668 - 5s/epoch - 233ms/step
Epoch 12/100
21/21 - 5s - loss: 0.2489 - val_loss: 0.2647 - 5s/epoch - 231ms/step
Epoch 13/100
21/21 - 5s - loss: 0.248

### Results

Running our Experiment yielded the following results:

| Configuration          | Min  | Max  | Mean  | St. Dev |
| ---------------------- | ---- | ---- | ----- | ------- |
| No Configuration       | 0.17 | 0.21 | 0.195 | 0.012   |
| Mirroring              | 0.17 | 0.22 | 0.191 | 0.015   |
| Rotation               | 0.19 | 0.24 | 0.214 | 0.014   |
| Shift                  | 0.21 | 0.25 | 0.231 | 0.015   |
| Brightness             | 0.08 | 0.13 | 0.111 | 0.019   |
| Zoom                   | 0.17 | 0.22 | 0.198 | 0.018   |

### Interpretation

Our results indicate that both rotation and shift seem to increase the hamming score and changing brightness dramatically decrased the score. Mirroring and zoom did not result in significant score changes.

Based on these results we identified the following follow-up questions:
* How do the results change if we activate all configurations except brightness?
* How do the results change if we activate only rotation and shift?
* Is the score increase from shift consistent? 

To answer these question the following additional data was recorded:

| Configuration          | Min  | Max  | Mean  | St. Dev |
| ---------------------- | ---- | ---- | ----- | ------- |
| No Configuration       | 0.17 | 0.21 | 0.195 | 0.012   |
| All except brightness  | 0.19 | 0.25 | 0.214 | 0.017   |
| Rotation + Shift       | 0.18 | 0.24 | 0.218 | 0.022   |
| Shift (rerun)          | 0.18 | 0.24 | 0.222 | 0.018 |

From these results we can see that activating all configurations except brightness yields the same quality as activating only brightness or only shift. The same holds for only activating rotation and shift. The effects of these preprocessing techniques do not seem to confound each other - or at least this is not visible in the relatively simple model we applied.

Rerunning sift also yielded a slighty lower result than before. For the future we decided to use rotation and shift for data preprocessing. 

As a side-node: our rerun of the shift preprocessing also shows that we should be careful even in interpreting the averaged results over ten runs. We will take this into account when doing bigger experiments by using more runs.


## Experiment 3: Increasing complexity
In this experiment we test whether we can improve our scores by increasing the network complexity. 

### Experiment Setup
For our experiment we use the configuration of the image data generator that performed best in Experiment 2 and test different complexity increases. We test:
- Adding convolutional layers
- increasing the number of convolutional patterns
- increasing the number of fully connected neurons
- adding more fully connected neurons. 

Due to technical difficulties, we had to restart the notebook environment for this test. This means, data recorded here is done with a different trainin / test split than the other experiments. The experiment itself has been executed entirely with the same training test split.

As in the previoius experiment, we use the CNN Factory and Multi Run Evaluation.
During our experiment we ran the cell below with different convigurations of the train data generator and recorded results in the cell below. 

In [5]:
train_datagen = ImageDataGenerator(
    rotation_range=30,
    #zoom_range = 0.20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    # brightness_range= [0.8, 1.2],
    #horizontal_flip=True,
    fill_mode="nearest"
)
train_generator = train_datagen.flow(x=X_train, y=y_train_prob)

cnn_builder = CNNBuilder(
    convolutional_layers=[64, 32, 16],
    fully_connected_layers=[100],
    in_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]),  # type: ignore
    out_shape=y_train.shape[1]  # type: ignore
)

evaluator = MultiRunEvaluation(model_creation=cnn_builder.build_model)
evaluator.evaluate(
    nr_runs=10,
    epochs=100,
    early_stopping_patience=5,
    train_generator= train_generator,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test
)
evaluator.print_metrics()


Running experiment 1 of, 10
Epoch 1/100


/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 13s - loss: 0.3390 - val_loss: 0.2854 - 13s/epoch - 630ms/step
Epoch 2/100
21/21 - 13s - loss: 0.2817 - val_loss: 0.2854 - 13s/epoch - 601ms/step
Epoch 3/100
21/21 - 13s - loss: 0.2813 - val_loss: 0.2841 - 13s/epoch - 624ms/step
Epoch 4/100
21/21 - 13s - loss: 0.2803 - val_loss: 0.2849 - 13s/epoch - 605ms/step
Epoch 5/100
21/21 - 13s - loss: 0.2810 - val_loss: 0.2845 - 13s/epoch - 605ms/step
Epoch 6/100
21/21 - 13s - loss: 0.2808 - val_loss: 0.2851 - 13s/epoch - 598ms/step
Epoch 7/100
21/21 - 13s - loss: 0.2806 - val_loss: 0.2826 - 13s/epoch - 616ms/step
Epoch 8/100
21/21 - 13s - loss: 0.2800 - val_loss: 0.2826 - 13s/epoch - 619ms/step
Epoch 9/100
21/21 - 13s - loss: 0.2809 - val_loss: 0.2821 - 13s/epoch - 607ms/step
Epoch 10/100
21/21 - 13s - loss: 0.2796 - val_loss: 0.2841 - 13s/epoch - 613ms/step
Epoch 11/100
21/21 - 13s - loss: 0.2788 - val_loss: 0.2821 - 13s/epoch - 599ms/step
Epoch 12/100
21/21 - 13s - loss: 0.2786 - val_loss: 0.2850 - 13s/epoch - 604ms/step
Epoch 13/100


/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 12s - loss: 0.3535 - val_loss: 0.2915 - 12s/epoch - 582ms/step
Epoch 2/100
21/21 - 12s - loss: 0.2829 - val_loss: 0.2894 - 12s/epoch - 592ms/step
Epoch 3/100
21/21 - 13s - loss: 0.2807 - val_loss: 0.2844 - 13s/epoch - 615ms/step
Epoch 4/100
21/21 - 14s - loss: 0.2791 - val_loss: 0.2819 - 14s/epoch - 653ms/step
Epoch 5/100
21/21 - 12s - loss: 0.2775 - val_loss: 0.2827 - 12s/epoch - 579ms/step
Epoch 6/100
21/21 - 14s - loss: 0.2764 - val_loss: 0.2800 - 14s/epoch - 657ms/step
Epoch 7/100
21/21 - 13s - loss: 0.2753 - val_loss: 0.2776 - 13s/epoch - 628ms/step
Epoch 8/100
21/21 - 13s - loss: 0.2735 - val_loss: 0.2747 - 13s/epoch - 599ms/step
Epoch 9/100
21/21 - 13s - loss: 0.2718 - val_loss: 0.2737 - 13s/epoch - 624ms/step
Epoch 10/100
21/21 - 13s - loss: 0.2703 - val_loss: 0.2751 - 13s/epoch - 637ms/step
Epoch 11/100
21/21 - 13s - loss: 0.2681 - val_loss: 0.2715 - 13s/epoch - 626ms/step
Epoch 12/100
21/21 - 12s - loss: 0.2658 - val_loss: 0.2720 - 12s/epoch - 589ms/step
Epoch 13/100


...

21/21 - 13s - loss: 0.3354 - val_loss: 0.2854 - 13s/epoch - 618ms/step
Epoch 2/100
21/21 - 13s - loss: 0.2820 - val_loss: 0.2834 - 13s/epoch - 621ms/step
Epoch 3/100
21/21 - 13s - loss: 0.2794 - val_loss: 0.2824 - 13s/epoch - 618ms/step
Epoch 4/100
21/21 - 14s - loss: 0.2798 - val_loss: 0.2819 - 14s/epoch - 665ms/step
Epoch 5/100
21/21 - 14s - loss: 0.2797 - val_loss: 0.2825 - 14s/epoch - 688ms/step
Epoch 6/100
21/21 - 13s - loss: 0.2797 - val_loss: 0.2826 - 13s/epoch - 642ms/step
Epoch 7/100
21/21 - 15s - loss: 0.2787 - val_loss: 0.2833 - 15s/epoch - 694ms/step
Epoch 8/100
21/21 - 13s - loss: 0.2775 - val_loss: 0.2818 - 13s/epoch - 628ms/step
Epoch 9/100
21/21 - 13s - loss: 0.2794 - val_loss: 0.2809 - 13s/epoch - 619ms/step
Epoch 10/100
21/21 - 13s - loss: 0.2768 - val_loss: 0.2803 - 13s/epoch - 625ms/step
Epoch 11/100
21/21 - 14s - loss: 0.2779 - val_loss: 0.2796 - 14s/epoch - 646ms/step
Epoch 12/100
21/21 - 13s - loss: 0.2779 - val_loss: 0.2800 - 13s/epoch - 630ms/step
Epoch 13/100


/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 13s - loss: 0.3376 - val_loss: 0.2890 - 13s/epoch - 598ms/step
Epoch 2/100
21/21 - 13s - loss: 0.2811 - val_loss: 0.2840 - 13s/epoch - 597ms/step
Epoch 3/100
21/21 - 12s - loss: 0.2798 - val_loss: 0.2848 - 12s/epoch - 561ms/step
Epoch 4/100
21/21 - 12s - loss: 0.2791 - val_loss: 0.2828 - 12s/epoch - 553ms/step
Epoch 5/100
21/21 - 12s - loss: 0.2789 - val_loss: 0.2818 - 12s/epoch - 572ms/step
Epoch 6/100
21/21 - 12s - loss: 0.2789 - val_loss: 0.2847 - 12s/epoch - 556ms/step
Epoch 7/100
21/21 - 12s - loss: 0.2782 - val_loss: 0.2787 - 12s/epoch - 557ms/step
Epoch 8/100
21/21 - 12s - loss: 0.2771 - val_loss: 0.2825 - 12s/epoch - 562ms/step
Epoch 9/100
21/21 - 12s - loss: 0.2771 - val_loss: 0.2810 - 12s/epoch - 559ms/step
Epoch 10/100
21/21 - 12s - loss: 0.2763 - val_loss: 0.2788 - 12s/epoch - 559ms/step
Epoch 11/100
21/21 - 12s - loss: 0.2772 - val_loss: 0.2810 - 12s/epoch - 553ms/step
Epoch 12/100
21/21 - 12s - loss: 0.2769 - val_loss: 0.2795 - 12s/epoch - 555ms/step
6/6 [========

/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 12s - loss: 0.3260 - val_loss: 0.2901 - 12s/epoch - 576ms/step
Epoch 2/100
21/21 - 12s - loss: 0.2836 - val_loss: 0.2833 - 12s/epoch - 576ms/step
Epoch 3/100
21/21 - 12s - loss: 0.2803 - val_loss: 0.2854 - 12s/epoch - 574ms/step
Epoch 4/100
21/21 - 12s - loss: 0.2795 - val_loss: 0.2826 - 12s/epoch - 580ms/step
Epoch 5/100
21/21 - 12s - loss: 0.2776 - val_loss: 0.2809 - 12s/epoch - 590ms/step
Epoch 6/100
21/21 - 12s - loss: 0.2771 - val_loss: 0.2820 - 12s/epoch - 577ms/step
Epoch 7/100
21/21 - 12s - loss: 0.2779 - val_loss: 0.2822 - 12s/epoch - 575ms/step
Epoch 8/100
21/21 - 12s - loss: 0.2775 - val_loss: 0.2795 - 12s/epoch - 579ms/step
Epoch 9/100
21/21 - 13s - loss: 0.2757 - val_loss: 0.2779 - 13s/epoch - 601ms/step
Epoch 10/100
21/21 - 12s - loss: 0.2750 - val_loss: 0.2794 - 12s/epoch - 592ms/step
Epoch 11/100
21/21 - 12s - loss: 0.2760 - val_loss: 0.2818 - 12s/epoch - 594ms/step
Epoch 12/100
21/21 - 12s - loss: 0.2741 - val_loss: 0.2778 - 12s/epoch - 583ms/step
Epoch 13/100


### Results


Running our Experiment yielded the following results:

| Test | Convolutional Layers | Fully Connected Layers | Min  | Max  | Mean  | St. Dev |
| ---- | -------------------- | ---------------------- | ---- | ----- | ---- | ------- |
| 1    | [16]                 | [100]                  | 0.20 | 0.23 | 0.217 | 0.012   |
| 2    | [32]                 | [100]                  | 0.16 | 0.22 | 0.201 | 0.021   |
| 3    | [32,16]              | [100]                  | 0.20 | 0.23 | 0.215 | 0.013   |
| 4    | [16]                 | [1000]                 | 0.15 | 0.23 | 0.204 | 0.023   |
| 5    | [16]                 | [100,100]              | 0.15 | 0.24 | 0.213 | 0.025   |
| 6    | [32,16]              | [100,100]              | 0.15 | 0.24 | 0.19  | 0.031   |
| 7    | [32,16]              | [20,20]                | 0.09 | 0.22 | 0.175 | 0.038   |
| 8    | [64,32,16]           | [100,100]              | 0.11 | 0.15 | 0.125 | 0.014   |
| 9    | [64,32,16]           | [100]                  | 0.10 | 0.23 | 0.173 | 0.03    |


### Interpretation

The numbers indicate that increasing the complexity did not improve results and in some cases significantly worsened the results. The best configurations we found were configurations 1, 3 and 5.
We will use these as starting points for future fine-tuning.



# Experiment 4: Network configuration in complex networks

In this experiment we rerun experiment 1 for a more complex network, to see if this changes the results and indicates that we should use any of the configurations.

## Experiment Setup: 

For experimentation we used configuration 6 from the previous experiment. While this configuration yielded worse scores than others, it is a combination of the two promising configurations 3 and 5. We did not use either of these two directly as we deem them to simple for this experiment. 


As in the previoius experiment, we use the CNN Factory and Multi Run Evaluation.
During our experiment we ran the cell below with different convigurations of the train data generator and recorded results in the cell below. 

In [6]:
train_datagen = ImageDataGenerator(
    rotation_range=30,
    #zoom_range = 0.20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    # brightness_range= [0.8, 1.2],
    #horizontal_flip=True,
    fill_mode="nearest"
)
train_generator = train_datagen.flow(x=X_train, y=y_train_prob)

cnn_builder = CNNBuilder(
    convolutional_layers=[32,16],
    fully_connected_layers=[100,100],
    in_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]),  # type: ignore
    out_shape=y_train.shape[1]  # type: ignore
)
cnn_builder.apply_regularization = False
cnn_builder.apply_dropout = False
cnn_builder.apply_batch_normalization = False
cnn_builder.weight_constraints = False

evaluator = MultiRunEvaluation(model_creation=cnn_builder.build_model)
evaluator.evaluate( nr_runs=10, 
                    epochs=100, 
                    early_stopping_patience=5, 
                    train_generator= train_generator, 
                    X_train=X_train, 
                    y_train=y_train, 
                    X_test=X_test,
                    y_test=y_test)
evaluator.print_metrics()


Running experiment 1 of, 10
Epoch 1/100


/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 7s - loss: 0.3422 - val_loss: 0.2840 - 7s/epoch - 342ms/step
Epoch 2/100
21/21 - 7s - loss: 0.2825 - val_loss: 0.2860 - 7s/epoch - 340ms/step
Epoch 3/100
21/21 - 7s - loss: 0.2806 - val_loss: 0.2841 - 7s/epoch - 347ms/step
Epoch 4/100
21/21 - 7s - loss: 0.2807 - val_loss: 0.2827 - 7s/epoch - 332ms/step
Epoch 5/100
21/21 - 7s - loss: 0.2785 - val_loss: 0.2833 - 7s/epoch - 328ms/step
Epoch 6/100
21/21 - 7s - loss: 0.2765 - val_loss: 0.2798 - 7s/epoch - 334ms/step
Epoch 7/100
21/21 - 7s - loss: 0.2748 - val_loss: 0.2811 - 7s/epoch - 329ms/step
Epoch 8/100
21/21 - 7s - loss: 0.2717 - val_loss: 0.2756 - 7s/epoch - 339ms/step
Epoch 9/100
21/21 - 7s - loss: 0.2704 - val_loss: 0.2732 - 7s/epoch - 346ms/step
Epoch 10/100
21/21 - 8s - loss: 0.2670 - val_loss: 0.2721 - 8s/epoch - 367ms/step
Epoch 11/100
21/21 - 8s - loss: 0.2637 - val_loss: 0.2688 - 8s/epoch - 367ms/step
Epoch 12/100
21/21 - 7s - loss: 0.2654 - val_loss: 0.2735 - 7s/epoch - 352ms/step
Epoch 13/100
21/21 - 7s - loss: 0.262

/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 8s - loss: 0.3344 - val_loss: 0.2860 - 8s/epoch - 361ms/step
Epoch 2/100
21/21 - 7s - loss: 0.2827 - val_loss: 0.2870 - 7s/epoch - 352ms/step
Epoch 3/100
21/21 - 7s - loss: 0.2817 - val_loss: 0.2861 - 7s/epoch - 341ms/step
Epoch 4/100
21/21 - 7s - loss: 0.2797 - val_loss: 0.2839 - 7s/epoch - 337ms/step
Epoch 5/100
21/21 - 7s - loss: 0.2783 - val_loss: 0.2806 - 7s/epoch - 352ms/step
Epoch 6/100
21/21 - 7s - loss: 0.2759 - val_loss: 0.2786 - 7s/epoch - 346ms/step
Epoch 7/100
21/21 - 7s - loss: 0.2734 - val_loss: 0.2757 - 7s/epoch - 349ms/step
Epoch 8/100
21/21 - 7s - loss: 0.2731 - val_loss: 0.2812 - 7s/epoch - 335ms/step
Epoch 9/100
21/21 - 8s - loss: 0.2715 - val_loss: 0.2750 - 8s/epoch - 368ms/step
Epoch 10/100
21/21 - 8s - loss: 0.2672 - val_loss: 0.2734 - 8s/epoch - 362ms/step
Epoch 11/100
21/21 - 7s - loss: 0.2659 - val_loss: 0.2702 - 7s/epoch - 351ms/step
Epoch 12/100
21/21 - 7s - loss: 0.2651 - val_loss: 0.2672 - 7s/epoch - 334ms/step
Epoch 13/100
21/21 - 7s - loss: 0.265

...

21/21 - 7s - loss: 0.3366 - val_loss: 0.2846 - 7s/epoch - 357ms/step
Epoch 2/100
21/21 - 7s - loss: 0.2809 - val_loss: 0.2899 - 7s/epoch - 348ms/step
Epoch 3/100
21/21 - 7s - loss: 0.2806 - val_loss: 0.2850 - 7s/epoch - 341ms/step
Epoch 4/100
21/21 - 8s - loss: 0.2794 - val_loss: 0.2807 - 8s/epoch - 385ms/step
Epoch 5/100
21/21 - 9s - loss: 0.2758 - val_loss: 0.2807 - 9s/epoch - 426ms/step
Epoch 6/100
21/21 - 8s - loss: 0.2745 - val_loss: 0.2785 - 8s/epoch - 392ms/step
Epoch 7/100
21/21 - 8s - loss: 0.2714 - val_loss: 0.2759 - 8s/epoch - 397ms/step
Epoch 8/100
21/21 - 8s - loss: 0.2697 - val_loss: 0.2741 - 8s/epoch - 398ms/step
Epoch 9/100
21/21 - 8s - loss: 0.2676 - val_loss: 0.2719 - 8s/epoch - 396ms/step
Epoch 10/100
21/21 - 8s - loss: 0.2666 - val_loss: 0.2711 - 8s/epoch - 372ms/step
Epoch 11/100
21/21 - 9s - loss: 0.2649 - val_loss: 0.2705 - 9s/epoch - 437ms/step
Epoch 12/100
21/21 - 8s - loss: 0.2649 - val_loss: 0.2696 - 8s/epoch - 382ms/step
Epoch 13/100
21/21 - 8s - loss: 0.263

/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


21/21 - 7s - loss: 0.3422 - val_loss: 0.2873 - 7s/epoch - 329ms/step
Epoch 2/100
21/21 - 7s - loss: 0.2823 - val_loss: 0.2844 - 7s/epoch - 322ms/step
Epoch 3/100
21/21 - 7s - loss: 0.2809 - val_loss: 0.2850 - 7s/epoch - 323ms/step
Epoch 4/100
21/21 - 7s - loss: 0.2808 - val_loss: 0.2838 - 7s/epoch - 321ms/step
Epoch 5/100
21/21 - 7s - loss: 0.2803 - val_loss: 0.2830 - 7s/epoch - 319ms/step
Epoch 6/100
21/21 - 7s - loss: 0.2808 - val_loss: 0.2829 - 7s/epoch - 323ms/step
Epoch 7/100
21/21 - 7s - loss: 0.2796 - val_loss: 0.2820 - 7s/epoch - 352ms/step
Epoch 8/100
21/21 - 7s - loss: 0.2792 - val_loss: 0.2829 - 7s/epoch - 339ms/step
Epoch 9/100
21/21 - 7s - loss: 0.2784 - val_loss: 0.2794 - 7s/epoch - 342ms/step
Epoch 10/100
21/21 - 7s - loss: 0.2771 - val_loss: 0.2782 - 7s/epoch - 348ms/step
Epoch 11/100
21/21 - 7s - loss: 0.2738 - val_loss: 0.2762 - 7s/epoch - 333ms/step
Epoch 12/100
21/21 - 7s - loss: 0.2730 - val_loss: 0.2743 - 7s/epoch - 355ms/step
Epoch 13/100
21/21 - 9s - loss: 0.268

/Users/ninovationlab/Projects/AI/05-Teaching-Materials/01-ML-Example-Project/evaluation/multi_run_evaluation.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(train_generator, epochs=epochs, validation_data=(X_test, y_test),callbacks = [early_stopping],verbose=verbose)


Epoch 1/100
21/21 - 8s - loss: 0.3374 - val_loss: 0.2861 - 8s/epoch - 396ms/step
Epoch 2/100
21/21 - 8s - loss: 0.2816 - val_loss: 0.2853 - 8s/epoch - 399ms/step
Epoch 3/100
21/21 - 9s - loss: 0.2799 - val_loss: 0.2834 - 9s/epoch - 415ms/step
Epoch 4/100
21/21 - 8s - loss: 0.2811 - val_loss: 0.2863 - 8s/epoch - 401ms/step
Epoch 5/100
21/21 - 9s - loss: 0.2796 - val_loss: 0.2834 - 9s/epoch - 439ms/step
Epoch 6/100
21/21 - 11s - loss: 0.2793 - val_loss: 0.2842 - 11s/epoch - 530ms/step
Epoch 7/100
21/21 - 10s - loss: 0.2784 - val_loss: 0.2869 - 10s/epoch - 453ms/step
Epoch 8/100
21/21 - 8s - loss: 0.2780 - val_loss: 0.2805 - 8s/epoch - 397ms/step
Epoch 9/100
21/21 - 9s - loss: 0.2755 - val_loss: 0.2788 - 9s/epoch - 406ms/step
Epoch 10/100
21/21 - 9s - loss: 0.2730 - val_loss: 0.2771 - 9s/epoch - 418ms/step
Epoch 11/100
21/21 - 9s - loss: 0.2709 - val_loss: 0.2755 - 9s/epoch - 424ms/step
Epoch 12/100
21/21 - 9s - loss: 0.2684 - val_loss: 0.2734 - 9s/epoch - 417ms/step
Epoch 13/100
21/21 - 

### Results

Running our Experiment yielded the following results:

| Configuration          | Min  | Max  | Mean  | St. Dev |
| ---------------------- | ---- | ---- | ----- | ------- |
| No Configuration (again!)      | 0.12 | 0.24 | 0.194 | 0.032   | 
| Regularization         | 0.13 | 0.19 | 0.175 | 0.018   |
| Dropout                | 0.1  | 0.23 | 0.177 | 0.047   |
| Batch Normalization    | 0.07 | 0.16 | 0.117 | 0.026   |
| Weight Constraints     | 0.16 | 0.21 | 0.184 | 0.016   |


### Interpretation.

The results achieved mirror those of Experiment 1. The configurations we tested seem to not make a big difference in changing the hamming score. Batch normalization in particular worsens the score significantly.

In the future, we will use regularization, dropout and weight constraints. Although we will do so more out of recognition of good practices as this experiment did not give cause to assume that they will improve matters drastically.

## Conclusions

The experiments performed in this notebook showed that we can improve the generalization of our models by using image generators to increase the variety of input images. The neural network configuration did not improve results but also did not worsen them.

As an outcome of the experiments, we derive the following knowledge for further experiments:
- Experiment 1 and 4: We will use regularization, dropount and weight constraints for our models.
- Experiment 2: We will use image generators with rotation and shift for image data preprocessing.
- Experiment 3: We will focus on models in the low-complexity end of CNNs. This means  we will test ...
    - no more than 2 convolutional layers
    - no more than 1000 Neurons in fully connected layers
    - no more than 2 fully connected layers.

We will use these for future experiments.

In addition, we noted that even with 10 runs, the numbers can differ significantly when rerunning an experiment. For this reason, we will use 100 runs in the future. 